In [ ]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np
import pyarrow as pa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [ ]:
# Pegar os dados dos jogos da NBA (2019-20 temporada até hoje)
date_from = '2019-10-22'
date_to = datetime.today().strftime('%Y-%m-%d')

gamefinder = leaguegamefinder.LeagueGameFinder(date_from_nullable=date_from, date_to_nullable=date_to)
games_df = gamefinder.get_data_frames()[0]

# Filtrar jogos da temporada regular 
# ( SEASON_ID:
#   1xxxx - Pré-Temporada
#   2xxxx - Temporada Regular
#   4xxxx - Playoffs
#   5xxxx - Play-In )
df = games_df[games_df['SEASON_ID'].astype(str).str.startswith('2')]

# Salva o dataframe em parquet
df.to_parquet('nba_games_2019_today.parquet', index=False)

In [ ]:
df = pd.read_parquet('nba_games_2019_today.parquet')

pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# Pré-processamento dos dados pra ML

# Formata a data pra datetime
df_clean = df.copy()
df_clean['GAME_DATE'] = pd.to_datetime(df_clean['GAME_DATE'])

# Cria a variável alvo: vitória ou derrota
df_clean['WIN'] = df_clean['WL'].apply(lambda x: 1 if x == 'W' else 0)

# Cria a variavel "HOME"
df_clean['HOME'] = df_clean['MATCHUP'].apply(lambda x: 1 if 'vs' in x else 0)

In [ ]:
df_clean.describe()

In [ ]:
# Modelo Random Forest com as features originais pra usar como baseline